# Import and definitions

In [1]:
from numpy import pi
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import qutip as qt

# Specify initial pulse and bounds

In [2]:
handmade_pulse = {
        'A' : 5,
        'L' : 150e-9,
        'alpha' : 1.001
}

pulse_bounds = { # Pairs of [min, max] values for each parameter
        'A' : [1, 10],
        'L' : [25e-9, 220e-9],
#        'alpha' : [0.999, 1.001]  # fix this parameter by commenting this line
}

initial_spread = { # Initial cloud will be sampled +- around these values 
        'A' : 0.2,
        'L' : 10e-9,
 #       'alpha' : 1.001
}

# Evaluation function

Provide an *evaluate_pulse* function and its helpers, specific to your experimental setup, that takes a several sets of parameters and provides a figure of merit for each.

In [3]:
def print_current_params(gate, q):
    """
    Print the current sample q to console in JSON format.
    """
    clear_output(wait=True)
    gate.print_pulse(q)

def evaluate_pulse(gate, samples):
    """
    Give a function that sends samples to the experiment 
    to be evaluated. Compute and return an infidelity for every
    point, that is to be minimized.
    
    Parameters
    ----------
    gate : C3 gate object
        Collection of parameters and helpers to represent a logical gate.
    samples : list
        List of points in parameter space to be evaluated. 
        samples = [s1, s2, s3, ...] where
        s1 = [param1, param2, param3, ...]

    Returns
    -------
    list
        Fidelity for each sample.
    """
    infidelities = None
    for q in samples:
        print_current_params(gate, q)
    return infidelities

# Import C3PO

In [4]:
from c3po.main.gate import Gate as gt
from c3po.fidelity.measurement import Experiment as exp_backend

In [5]:
CP_gate = gt('qubit_12', T_final = 250e-9)
CP_gate.env_shape = 'flat' # Pulses are specified by a list of parameters without nested structure.
CP_gate.set_parameters('initial', handmade_pulse)
CP_gate.set_bounds(pulse_bounds)

fridge = exp_backend(evaluate_pulse)

# Calibration

In [6]:
opts = {
    'CMA_stds': initial_spread,
    'ftarget' : 1e-4,
    'popsize' : 20
}

fridge.calibrate(CP_gate, opts)

{
    "A": 5.068838993668959,
    "L": 1.4389762680311458e-07,
    "alpha": 1.001
}


TypeError: object of type 'NoneType' has no len()